In [2]:
%pip install -q --disable-pip-version-check semantic-link-labs

Note: you may need to restart the kernel to use updated packages.


# Setup Libraries and Parameters for Notebook

In [3]:
import sempy_labs as labs
from sempy import fabric
import pandas as pd
import json
import time
import uuid
from sempy_labs.tom._model import TOMWrapper, connect_semantic_model

from sempy_labs._helper_functions import (
    format_dax_object_name,
    generate_guid,
    _make_list_unique,
    resolve_dataset_name_and_id,
    resolve_workspace_name_and_id,
    _base_api,
    resolve_workspace_id,
    resolve_item_id,
    resolve_lakehouse_id,
    resolve_lakehouse_name_and_id
)

LakehouseName = "BigData"
SemanticModelName = f"{LakehouseName}_model"
ClonedModelName = SemanticModelName + "_clone"
workspace = None

with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=True) as tom:
    import Microsoft.AnalysisServices.Tabular as TOM
    import Microsoft.AnalysisServices

(workspace_name, workspace_id) = resolve_workspace_name_and_id(workspace)
(lakehouse_name, lakehouse_id) = resolve_lakehouse_name_and_id(lakehouse=LakehouseName, workspace=workspace)


# Clone AdventureWorks semantic model

In [5]:
#Clear any existing cloned model if re-running
df = fabric.list_items()
if ClonedModelName in df.values:
    model_id = df.at[df[df['Display Name'] == ClonedModelName].index[0], 'Id']
    fabric.delete_item(model_id)
    print("Cloned model deleted")

with labs.tom.connect_semantic_model(dataset=SemanticModelName, readonly=False) as tom:
    newDB = tom._tom_server.Databases.GetByName(SemanticModelName).Clone()
    newModel = tom._tom_server.Databases.GetByName(SemanticModelName).Model.Clone()
    newDB.Name = ClonedModelName
    newDB.ID = str(uuid.uuid4())
    #newDB.Model = newModel
    newModel.CopyTo(newDB.Model)
    tom._tom_server.Databases.Add(newDB)

    newDB.Update(Microsoft.AnalysisServices.UpdateOptions.ExpandFull)

# Frame the cloned model

In [7]:
labs.refresh_semantic_model(dataset=ClonedModelName)

⌛ Refresh of the 'BigData_model_clone' semantic model within the 'Philtest5' workspace is in progress...
🟢 Refresh 'full' of the 'BigData_model_clone' semantic model within the 'Philtest5' workspace is complete.


# Check what version of Direct Lake is being used

## Sql.Database    = DirectLake over SQL

## Azure.Lakehouse = DirectLake over One Lake

In [8]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)

let
	database = Sql.Database("krrvjodmlo3ehogm2woebho6je-l3sxbm65nd7uvf65fpoxtd4ouq.datawarehouse.fabric.microsoft.com", "30625321-400e-4963-bed7-d68a04c42289")
in
	database


# Show storage mode for each table in Cloned model

In [9]:
objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

# Try to convert Direct Lake table to Import
#### Will fail if Direct Lake over SQL

In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    tom.convert_direct_lake_to_import(
        table_name="dim_Date" ,
        entity_name="dim_Date" ,
        source="BigData",
        source_type = "Lakehouse"
    )

# Convert cloned model to Direct Lake over One Lake

In [11]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:

    #Convert import tables to Direct Lake
    for t in tom.model.Tables:
        for p in t.Partitions:
            if(p.Mode==TOM.ModeType.Import):
                t.Partitions.Remove(p)
                tom.add_entity_partition(table_name=t.Name,entity_name=t.Name)
                print(f"Table {t.Name} converted")
            p.Source.SchemaName=None

    for e in tom.model.Expressions:
        e.Expression = f"""
        let
            Source = AzureStorage.DataLake("https://northcentralus-onelake.dfs.fabric.microsoft.com/{workspace_id}/{lakehouse_id}", [HierarchicalNavigation=true])
        in
            Source"""
        
print("Converted Import tables back to Direct Lake (on One Lake)")

Converted Import tables back to Direct Lake (on One Lake)


# Try to convert Direct Lake table to Import (_attempt 2_)
## Should work this time now model is Direct Lake over One Lake

In [12]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    tom.convert_direct_lake_to_import(
        table_name="dim_Date" ,
        entity_name="dim_Date" ,
        source="BigData",
        source_type = "Lakehouse"
    )

🟢 The 'dim_Date' table has been converted to Import mode.


# Show storage mode for each table

In [13]:
objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

# <mark>SET CREDENTIALS</mark>

# Refresh composite model so import table gets hydrated

In [16]:
labs.refresh_semantic_model(dataset=ClonedModelName,tables=["dim_Date"])

⌛ Refresh of the 'BigData_model_clone' semantic model within the 'Philtest5' workspace is in progress...
🟢 Refresh 'full' of the 'BigData_model_clone' semantic model within the 'Philtest5' workspace is complete.


# Show what version of Direct Lake is being used

In [17]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)


        let
            Source = AzureStorage.DataLake("https://northcentralus-onelake.dfs.fabric.microsoft.com/b370e55e-68dd-4aff-97dd-2bdd798f8ea4/c32095f5-be09-4918-a6ac-e48e47e47825", [HierarchicalNavigation=true])
        in
            Source


# Run query on 1Bln Row

In [23]:
df=fabric.evaluate_dax(
    dataset=ClonedModelName,
    dax_string="""
        EVALUATE
        SUMMARIZECOLUMNS(
               
                dim_Date[FirstDateofMonth] ,
                "Count of Transactions" , COUNTROWS(fact_myevents_1bln)
        )
        ORDER BY [FirstDateofMonth]
        """

    )

FabricAdomdException: An error occurred when running AdomdCommand. AdomdCommandActivityId: '13a08014-820e-42fe-8e71-dcde2082fa05'

Caused by AdomdErrorResponseException:
An unexpected exception occurred.
An unexpected exception occurred.

Technical Details:
RootActivityId: 13340122-96d7-421b-a5da-b49b816dc582
Date (UTC): 6/11/2025 11:44:50 PM
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.CheckForSoapFault(XmlReader reader, XmlaResult xmlaResult, Boolean throwIfError)
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.CheckForError(XmlReader reader, XmlaResult xmlaResult, Boolean throwIfError)
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.SendMessage(Boolean endReceivalIfException, Boolean readSession, Boolean readNamespaceCompatibility)
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.ExecuteStatement(String statement, IDictionary connectionProperties, IDictionary commandProperties, IDataParameterCollection parameters, Boolean isMdx)
   at Microsoft.Anal

# Run qury on 2Bln Row

# Convert cloned model to Direct Lake over SQL Endpoint

In [73]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:

    #Convert import tables to Direct Lake
    for t in tom.model.Tables:
        for p in t.Partitions:
            if(p.Mode==TOM.ModeType.Import):
                t.Partitions.Remove(p)
                tom.add_entity_partition(table_name=t.Name,entity_name=t.Name)
                print(f"Table {t.Name} converted")
            p.Source.SchemaName=None

    #Switch Model to Direct Lake over SQL
    for e in tom.model.Expressions:
        e.Expression = f"""
        let
            Source = Sql.Database("krrvjodmlo3ehogm2woebho6je-zp5iwh24wstujjehvqjfvnebvq.datawarehouse.fabric.microsoft.com", "7c63a301-c62e-408e-9c5e-b7b12b41895a")
        in
            Source"""

        # if e.Name == "DirectLake - AdventureWorks" :
        #     tom.model.Expressions.Remove(e)

print("Converted to Direct Lake over SQL")

StatementMeta(, 3e13bf92-0430-4b5c-ac73-c9d5f7d2363b, 80, Finished, Available, Finished)

Converted to Direct Lake over SQL


In [ ]:
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for e in tom.model.Expressions:
        print(e.Expression)

StatementMeta(, de1b840a-ebca-4a96-bd96-1d4e75e1e14d, 135, Finished, Available, Finished)


        let
            Source = AzureStorage.DataLake("https://northcentralus-onelake.dfs.fabric.microsoft.com/1f8bfacb-b45c-44a7-a487-ac125ab481ac/424a14f9-4a1b-468e-af8d-65add767ad40", [HierarchicalNavigation=true])
        in
            Source


# Show storage mode for each table

In [1]:
import pandas as pd

objects = {}
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    for t in tom.model.Tables:
        #print(t.Name)
        for p in t.Partitions:
            #print(p.Mode)
            objects[t.Name] = str(p.Mode)
 
df=pd.DataFrame([objects])
display(df)

NameError: name 'labs' is not defined

# Show TMSL code for cloned model

In [24]:
import json
with labs.tom.connect_semantic_model(dataset=ClonedModelName, readonly=False) as tom:
    x= tom.get_bim()

    formatted_json = json.dumps(x, indent=4)
    print(formatted_json)

{
    "name": "BigData_model_clone",
    "id": "59e9c1fd-0ec5-443e-83fb-6d816446a54e",
    "compatibilityLevel": 1605,
    "model": {
        "culture": "en-US",
        "collation": "Latin1_General_100_BIN2_UTF8",
        "dataAccessOptions": {
            "legacyRedirects": true,
            "returnErrorValuesAsNull": true
        },
        "defaultPowerBIDataSourceVersion": "powerBI_V3",
        "sourceQueryCulture": "en-US",
        "tables": [
            {
                "name": "fact_myevents_1bln",
                "lineageTag": "db08dfe4-ab35-4d61-882a-c84b595b5c47",
                "sourceLineageTag": "[dbo].[fact_myevents_1bln]",
                "columns": [
                    {
                        "name": "DateKey",
                        "dataType": "dateTime",
                        "isHidden": true,
                        "sourceColumn": "DateKey",
                        "lineageTag": "81e4f4de-c630-4ec8-ad0f-66c9ea8e9968"
                    },
               